**合併數據**

In [1]:
import pandas as pd
import glob
import os

folder_path = './Training data/'
files = glob.glob(os.path.join(folder_path, '*.csv'))
data = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.30,652.92,0.12
1,10,2024-03-01 17:14:47.000,0.0,1017.48,15.66,94.04,682.50,0.12
2,10,2024-03-01 17:15:47.000,0.0,1017.47,15.74,94.10,750.00,0.14
3,10,2024-03-01 17:16:47.000,0.0,1017.46,15.78,94.09,738.33,0.14
4,10,2024-03-01 17:17:47.000,0.0,1017.49,15.80,94.08,660.83,0.12
...,...,...,...,...,...,...,...,...
1375023,9,2024-10-18 14:27:02.000,0.0,1006.65,33.09,52.98,5280.83,6.71
1375024,9,2024-10-18 14:28:02.000,0.0,1006.67,33.07,53.42,4965.00,5.74
1375025,9,2024-10-18 14:29:02.000,0.0,1006.68,32.92,53.73,4709.17,5.13
1375026,9,2024-10-18 14:30:02.000,0.0,1006.70,32.92,53.55,4480.00,4.69


**合併地點與時間**

In [2]:
data['DateTime'] = pd.to_datetime(data['DateTime'])
data = data.sort_values(by=['LocationCode', 'DateTime'])
#data['Combined'] =  pd.to_datetime(data['DateTime']).dt.strftime('%Y%m%d%H%M') + data['LocationCode'].apply(lambda x: f'{x:02}')
data['DateLoc'] = data['DateTime'].dt.strftime('%Y%m%d') + data['LocationCode'].apply(lambda x : f'{x:02}')
data.to_csv('combined.csv', index=False)

data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),DateLoc
656248,1,2024-01-01 06:31:08,0.0,1016.50,17.5,86.8,25.00,0.00,2024010101
656249,1,2024-01-01 06:32:08,0.0,1016.53,17.5,86.7,28.33,0.00,2024010101
656250,1,2024-01-01 06:33:08,0.0,1016.57,17.5,86.7,32.50,0.00,2024010101
656251,1,2024-01-01 06:34:08,0.0,1016.58,17.5,86.7,39.17,0.00,2024010101
656252,1,2024-01-01 06:35:08,0.0,1016.59,17.5,86.7,45.83,0.00,2024010101
...,...,...,...,...,...,...,...,...,...
656243,17,2024-07-12 15:26:46,0.0,1004.05,36.5,65.4,12315.00,35.91,2024071217
656244,17,2024-07-12 15:27:47,0.0,1004.08,36.5,64.7,12692.50,38.23,2024071217
656245,17,2024-07-12 15:28:47,0.0,1004.10,36.5,64.5,12960.00,39.95,2024071217
656246,17,2024-07-12 15:29:47,0.0,1004.11,36.4,64.9,12751.67,38.51,2024071217


**讀取日期**

In [3]:
upload_file = 'upload.csv'
target_data = pd.read_csv(upload_file)
target_set = set()

target_data['Date'] = target_data['序號'].astype(str).str[:8]
target_data['LocationCode'] = target_data['序號'].astype(str).str[12:14]

target_data['DateLoc'] = target_data['Date'] + target_data['LocationCode']
target_set.update(target_data['DateLoc'].unique())
len(target_set)

200

**分割題目**

In [4]:
train_data = data[~data['DateLoc'].isin(target_set)]
input_data = data[data['DateLoc'].isin(target_set)]
input_data.to_csv('input.csv', index=False)
train_data.to_csv('train.csv', index=False)

**異常值處理**

**SunLight(Lux)**

In [5]:
from sklearn.model_selection import train_test_split
filtered_data = train_data[train_data['Sunlight(Lux)'] != 117758.2]
# 特徵與標籤
X = filtered_data[['Power(mW)']]  # 發電量 (特徵)
y = filtered_data['Sunlight(Lux)']  # 光照度 (目標)
# 分割訓練與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.linear_model import LinearRegression

# 建立線性回歸模型
model = LinearRegression()

# 訓練模型
model.fit( X_train,y_train )

# 獲取模型係數與截距
print(f"模型係數: {model.coef_[0]}")  # 光照度對發電量的影響係數
print(f"模型截距: {model.intercept_}")  # 發電量基線


模型係數: 55.60570241742195
模型截距: 7422.522839557063


In [7]:
from sklearn.metrics import mean_squared_error, r2_score

# 預測
y_pred = model.predict(X_test)

# 計算誤差與解釋力
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 99281509.20590872
R2: 0.8667737009319059


In [8]:
# 用回歸模型補充異常光照度對應的發電量
data.loc[data['Sunlight(Lux)'] == 117758.2, 'Sunlight(Lux)'] = \
    model.predict(data[data['Sunlight(Lux)'] == 117758.2][['Power(mW)']])

In [9]:
data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),DateLoc
656248,1,2024-01-01 06:31:08,0.0,1016.50,17.5,86.8,25.00,0.00,2024010101
656249,1,2024-01-01 06:32:08,0.0,1016.53,17.5,86.7,28.33,0.00,2024010101
656250,1,2024-01-01 06:33:08,0.0,1016.57,17.5,86.7,32.50,0.00,2024010101
656251,1,2024-01-01 06:34:08,0.0,1016.58,17.5,86.7,39.17,0.00,2024010101
656252,1,2024-01-01 06:35:08,0.0,1016.59,17.5,86.7,45.83,0.00,2024010101
...,...,...,...,...,...,...,...,...,...
656243,17,2024-07-12 15:26:46,0.0,1004.05,36.5,65.4,12315.00,35.91,2024071217
656244,17,2024-07-12 15:27:47,0.0,1004.08,36.5,64.7,12692.50,38.23,2024071217
656245,17,2024-07-12 15:28:47,0.0,1004.10,36.5,64.5,12960.00,39.95,2024071217
656246,17,2024-07-12 15:29:47,0.0,1004.11,36.4,64.9,12751.67,38.51,2024071217


# Model Training

**use Lstm and pre-processed data to train our model**

In [10]:
features = ['Pressure(hpa)', 'Temperature(°C)', 
            'Humidity(%)', 'Sunlight(Lux)']
X = train_data[features]
y = train_data['Power(mW)']

In [11]:
import numpy as np

def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X.iloc[i:i+time_steps].values)
        y_seq.append(y.iloc[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 10  # 每次輸入的時間步長
X_seq, y_seq = create_sequences(X, y, time_steps)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, activation='relu', input_shape=(time_steps, len(features))),
    Dense(1)  # 預測單一數值
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=256, validation_split=0.2)

Epoch 1/3
3330/3330 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 33517.9688 - mae: 75.1424 - val_loss: 33846.9805 - val_mae: 74.1839
Epoch 2/3
3330/3330 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 33286.6445 - mae: 74.9262 - val_loss: 34229.2031 - val_mae: 76.6435
Epoch 3/3
3330/3330 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 33328.2578 - mae: 74.7084 - val_loss: 33621.0703 - val_mae: 75.4844


In [84]:
model.save('./models/trained_model.h5') 

In [85]:
predictions = model.predict(X_test)
predictions_df = pd.DataFrame(predictions, columns=['Prediction'])

# 將 DataFrame 存成 CSV
predictions_df.to_csv('predictions.csv', index=False)

print("預測結果已成功儲存到 predictions.csv！")
print(predictions_df['Prediction'].value_counts())


8323/8323 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step
預測結果已成功儲存到 predictions.csv！
Prediction
8.544649      5
8.279299      5
6.663820      5
7.284914      4
8.498110      4
             ..
2.389741      1
13.562197     1
323.488129    1
34.428772     1
84.435883     1
Name: count, Length: 253144, dtype: int64


In [86]:
import pandas as pd

results = pd.DataFrame({
    'Id': ['202402180900{:02d}'.format(i) for i in range(1, len(predictions) + 1)],
    'Power(mW)': predictions.flatten().round(2)
})

results.to_csv('upload(test).csv', index=False)

In [87]:
import pandas as pd
import os

# 讀取數據
df = pd.read_csv('input.csv')

# 設定儲存資料夾路徑
output_folder = './inputdata'
# 如果資料夾不存在，則創建資料夾
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 根據日期分割數據並儲存為 CSV 文件
index = 0
for date, group in df.groupby('DateLoc'):
    # 設定每個分組儲存的文件名稱
    filename = f"{date}.csv"
    file_path = os.path.join(output_folder, filename)
    
    # 儲存該分組的 CSV 文件
    group.to_csv(file_path, index=False)
    print(f"Saved: {index} {filename}")
    index +=1

Saved: 0 2024010601.csv
Saved: 1 2024011101.csv
Saved: 2 2024011701.csv
Saved: 3 2024011901.csv
Saved: 4 2024012317.csv
Saved: 5 2024012408.csv
Saved: 6 2024012502.csv
Saved: 7 2024012802.csv
Saved: 8 2024012808.csv
Saved: 9 2024012917.csv
Saved: 10 2024020408.csv
Saved: 11 2024020717.csv
Saved: 12 2024021301.csv
Saved: 13 2024021302.csv
Saved: 14 2024021413.csv
Saved: 15 2024021515.csv
Saved: 16 2024021517.csv
Saved: 17 2024021808.csv
Saved: 18 2024021815.csv
Saved: 19 2024021817.csv
Saved: 20 2024022113.csv
Saved: 21 2024022401.csv
Saved: 22 2024022417.csv
Saved: 23 2024022514.csv
Saved: 24 2024022601.csv
Saved: 25 2024022602.csv
Saved: 26 2024022714.csv
Saved: 27 2024030508.csv
Saved: 28 2024030517.csv
Saved: 29 2024030710.csv
Saved: 30 2024030909.csv
Saved: 31 2024030917.csv
Saved: 32 2024031010.csv
Saved: 33 2024031117.csv
Saved: 34 2024031409.csv
Saved: 35 2024031413.csv
Saved: 36 2024031415.csv
Saved: 37 2024031512.csv
Saved: 38 2024031610.csv
Saved: 39 2024031613.csv
Saved: 40 

In [91]:
import pandas as pd
import numpy as np
import os

# 全局 sequence 初始化為 None
sequence = None

# 模型預測輸出限制範圍（避免極端值）
def clamp(value, min_value, max_value):
    return max(min(value, max_value), min_value)

# 平滑輸入序列
def smooth_sequence(sequence, alpha=0.1):
    smoothed_seq = np.copy(sequence)
    for i in range(1, len(sequence)):
        smoothed_seq[i] = alpha * sequence[i] + (1 - alpha) * smoothed_seq[i - 1]
    return smoothed_seq

# 預測特定序列的瓦數
def predict_for_dataloc(sequence):
    X_input = np.expand_dims(sequence, axis=0).astype(np.float32)
    predicted_power = model.predict(X_input)  # 假設 model 已加載
    print(predicted_power)
    return predicted_power

# 預測特定時間範圍內的平均瓦數
def predict_minute_power(targetdata, dateloc, start_time, end_time):
    global sequence
    filtered_df = targetdata[targetdata['DateLoc'] == dateloc]
    filtered_df = filtered_df.assign(序號=pd.to_datetime(filtered_df['序號'], format='%Y%m%d%H%M%S'))
    filtered_df = filtered_df[(filtered_df['序號'] >= start_time) & (filtered_df['序號'] <= end_time)]

    if not filtered_df.empty:
        minute_predictions = []
        current_time = start_time

        while current_time <= end_time:
            # 預測當前瓦數並記錄
            predicted_power = predict_for_dataloc(sequence)
            minute_predictions.append(predicted_power)

            # 更新全局 sequence
            sequence = np.roll(sequence, shift=-1, axis=0)
            sequence[-1] = predicted_power

            # 平滑 sequence（可選）
            sequence = smooth_sequence(sequence)

            current_time += pd.Timedelta(minutes=1)

        # 計算平均瓦數
        final_avg_power = np.mean(minute_predictions)
    else:
        final_avg_power = 0.0

    return round(final_avg_power, 2)

# 生成預測結果
def generate_predictions(target_data, features, output_file):
    global sequence
    results = []
    old = None
    df = None

    for _, row in target_data.iterrows():
        target_date = row['Date']
        target_time = f"{str(row['序號'])[8:10]}:{str(row['序號'])[10:12]}:00"
        start_time = pd.Timestamp(f"{target_date} {target_time}")
        end_time = start_time + pd.Timedelta(minutes=9, seconds=59)

        # 每日首次初始化 sequence
        if old != target_date:
            file_path = f"inputdata/{row['DateLoc']}.csv"
            if not os.path.exists(file_path):
                print(f"文件 {file_path} 不存在，跳過處理。")
                continue

            # 加載當前日期數據
            df = pd.read_csv(file_path)
            df['DateTime'] = pd.to_datetime(df['DateTime'])

            # 初始化 sequence 為有效數據的最後一段
            valid_data = df[df['DateTime'] <= start_time]
            sequence = valid_data[features].tail(len(features)).to_numpy()

            # 檢查 sequence 初始化有效性
            if sequence.shape[0] < len(features):
                print("序列長度不足，填充零值初始化。")
                sequence = np.zeros((len(features),))

        # 預測當前時間範圍內的平均瓦數
        avg_power = predict_minute_power(target_data, row['DateLoc'], start_time, end_time)
        results.append({'序號': row['序號'], '答案': f"{avg_power:.2f}"})
        print(f"序號: {row['序號']}, 預測瓦數: {avg_power:.2f}")

        # 更新日期標記
        old = target_date

    # 保存結果至 CSV
        results_df = pd.DataFrame(results)
        results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"預測結果已成功儲存到 {output_file}！")

# 假設 target_data 和 features 已定義
generate_predictions(target_data, features, 'upload_predictions.csv')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[240.92975]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[248.55568]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[252.76161]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[254.38025]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[254.99115]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[[255.31067]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[255.54636]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[255.758]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[[255.96278]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[256.1657]]
序號: 20240117090001, 預測瓦數: 253.04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[256.3683]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[256.5707]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[256.7733]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[256.97595]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[257.17892]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[257.38196]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[257.58505]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[257.78842]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

KeyboardInterrupt: 